# Bangla NER

The Bangla NER model build using Spacy. This model was train for person extraction purposes.

Uses Dataset are :

In [ ]:
!pip install spacy==3.6.1


# Write the base config file to directory. For more information please check    [spacy](https://spacy.io/usage/training)

In [ ]:
%%writefile base_config.cfg
# This is an auto-generated partial config. To use it with 'spacy train'
# you can run spacy init fill-config to auto-fill all default settings:
# python -m spacy init fill-config ./base_config.cfg ./config.cfg
[paths]
train = null
dev = null
vectors = null
[system]
gpu_allocator = null

[nlp]
lang = "en"
pipeline = ["tok2vec","ner"]
batch_size = 1000

[components]

[components.tok2vec]
factory = "tok2vec"

[components.tok2vec.model]
@architectures = "spacy.Tok2Vec.v2"

[components.tok2vec.model.embed]
@architectures = "spacy.MultiHashEmbed.v2"
width = ${components.tok2vec.model.encode.width}
attrs = ["NORM", "PREFIX", "SUFFIX", "SHAPE"]
rows = [5000, 1000, 2500, 2500]
include_static_vectors = false

[components.tok2vec.model.encode]
@architectures = "spacy.MaxoutWindowEncoder.v2"
width = 96
depth = 4
window_size = 1
maxout_pieces = 3

[components.ner]
factory = "ner"

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = true
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
width = ${components.tok2vec.model.encode.width}

[corpora]

[corpora.train]
@readers = "spacy.Corpus.v1"
path = ${paths.train}
max_length = 0

[corpora.dev]
@readers = "spacy.Corpus.v1"
path = ${paths.dev}
max_length = 0

[training]
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"

[training.optimizer]
@optimizers = "Adam.v1"

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2

[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001

[initialize]
vectors = ${paths.vectors}

Writing base_config.cfg


# Convert Base config to training config file

In [ ]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

2023-12-07 18:00:10.679830: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 18:00:10.679903: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 18:00:10.679943: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 18:00:10.689402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 18:00:11.978788: W tensorflow/compiler/

## Data conversion to spacy format. Here we convert BLIOU form to spacy format.

BLIOU and BIO format are similar type data format for NER.

BLIOU data format:

    B = Begin

    L = Last

    I = Inside

    O = Outside

    U = Unique

IOB data format:

    I = Inside

    O = Outside

    B = Begin


In [ ]:
# !pip install wget
# import wget

# # ! wget -r "https://drive.google.com/file/d/1-szaaF__gc0n8_X2grhVWbRLRKo_Havb/view?usp=sharing"

# !wget --no-check-certificate "https://drive.google.com/file/d/1-szaaF__gc0n8_X2grhVWbRLRKo_Havb/view?usp=sharing" -O ner_data.zip


--2023-12-07 18:15:57--  https://drive.google.com/file/d/1-szaaF__gc0n8_X2grhVWbRLRKo_Havb/view?usp=sharing
Resolving drive.google.com (drive.google.com)... 74.125.139.139, 74.125.139.113, 74.125.139.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.139.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘ner_data.zip’

ner_data.zip            [ <=>                ]  82.63K  --.-KB/s    in 0.004s  

2023-12-07 18:15:57 (20.1 MB/s) - ‘ner_data.zip’ saved [84610]



Googe Drive dataset link directory : https://drive.google.com/file/d/1-twRMl-2XfNrrchxqMW0hWaa9mopJGrw/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/bangla_ner_bliou.zip ./

In [ ]:
!unzip bangla_ner_bliou.zip

Archive:  bangla_ner_bliou.zip
   creating: bangla_ner_bliou/
  inflating: bangla_ner_bliou/train.json  
  inflating: bangla_ner_bliou/val.json  


# Convert bangla ber Bliou data format to spacy format

In [ ]:
!python -m spacy convert bangla_ner_bliou/train.json ./bangla_ner_bliou
!python -m spacy convert bangla_ner_bliou/val.json ./bangla_ner_bliou

2023-12-07 18:21:32.031072: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 18:21:32.031160: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 18:21:32.031208: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 18:21:32.044615: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 18:21:33.929505: W tensorflow/compiler/

# Training The Spacy Name Entity Recognition(NER) Model

In [ ]:
%%writefile train.sh
python -m spacy train config.cfg \
    --output ./models \
    --paths.train ./bangla_ner_bliou/train.spacy \
    --paths.dev ./bangla_ner_bliou/val.spacy

Overwriting train.sh


In [ ]:
!bash train.sh

2023-12-07 18:26:25.202401: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-07 18:26:25.202475: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-07 18:26:25.202521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-07 18:26:25.214359: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-07 18:26:27.048240: W tensorflow/compiler/

# Evaluation

# Inference

In [ ]:
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("./models/model-best")

text_list = [
    ": আল স্কিনিয়ার গিটার, পিয়ানো, ভোকাল, মগ সিনথেসাইজার",
    ]
for text in text_list:
    doc = nlp(text)

    print(f"Input: {text}")
    for entity in doc.ents:
        print(f"Entity: {entity.text}, Label: {entity.label_}")
    print("---")


Input: : আল স্কিনিয়ার গিটার, পিয়ানো, ভোকাল, মগ সিনথেসাইজার
Entity: আল স্কিনিয়ার, Label: PER
---
